### Membuat koneksi ke database
Library `SQLite` adalah salah satu library dalam python untuk untuk berinteraksi dengan SQL.

- Function `connect()` akan membuat koneksi dengan database, apabila database itu belum dibuat maka otomatis dibuat.
- Method  `cursor()` digunakan untuk membuat objek cursor yang memungkinkan kita menjalankan perintah SQL dan mengambil hasil query dari database.

In [1]:
import sqlite3

# Membuka koneksi ke file database SQLite
conn = sqlite3.connect("hotel.db")
cur = conn.cursor()

### Buat Tabel `Rooms` dan `Bookings`
- `Rooms` berisi data ruangan hotel
- `Bookings` berisi data pemesanan ruangan

Method `executescript()` digunakan untuk lebih dari satu menjalankan perintah SQL dalam sekali jalan.

In [2]:
cur.executescript("""
CREATE TABLE IF NOT EXISTS Rooms (
    room_id INTEGER PRIMARY KEY AUTOINCREMENT,
    number TEXT NOT NULL,
    type TEXT NOT NULL,
    price REAL NOT NULL
);

CREATE TABLE IF NOT EXISTS Bookings (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    guest_name TEXT NOT NULL,
    room_id INTEGER NOT NULL,
    check_in DATE NOT NULL,
    check_out DATE NOT NULL,
    FOREIGN KEY (room_id) REFERENCES Rooms(room_id)
);
""")
conn.commit()

# Cek apakah tabel Rooms dan Bookings sudah ada
room = cur.execute("PRAGMA table_info('Rooms');")
for col in room:
    print(col)
print("-----------------------")
bookings = cur.execute("PRAGMA table_info('Bookings');")
for col in bookings:
    print(col)

(0, 'room_id', 'INTEGER', 0, None, 1)
(1, 'number', 'TEXT', 1, None, 0)
(2, 'type', 'TEXT', 1, None, 0)
(3, 'price', 'REAL', 1, None, 0)
-----------------------
(0, 'booking_id', 'INTEGER', 0, None, 1)
(1, 'guest_name', 'TEXT', 1, None, 0)
(2, 'room_id', 'INTEGER', 1, None, 0)
(3, 'check_in', 'DATE', 1, None, 0)
(4, 'check_out', 'DATE', 1, None, 0)


### Tambahkan Data `Rooms`
Method `executemany()` digunakan untuk mengeksekusi perintah SQL yang sama berkali-kali dengan data yang berbeda.

In [3]:
rooms_data = [
    ("101", "Single", 500000),
    ("102", "Double", 750000),
    ("201", "Suite", 1500000),
    ("202", "Single", 500000),
    ("203", "Deluxe", 1700000),
]
cur.executemany("INSERT INTO Rooms (number, type, price) VALUES (?, ?, ?)", rooms_data)
conn.commit()

### Menambahkan data `Bookings`
Menjalankan dengan method `execute()` berarti hanya mengeksekusi satu perintah SQL

In [4]:
cur.execute(
    "INSERT INTO Bookings (guest_name, room_id, check_in, check_out) VALUES (?, ?, ?, ?)",
    ("Alice", 1, "2025-04-25", "2025-04-28")
)
conn.commit()

### Mengambil 1 `Bookings`
Methode `fetchone()` berarti ia hanya mengambil 1 baris data dari perintah SQL

In [5]:
cur.execute("SELECT * FROM Bookings WHERE guest_name = 'Alice';")
cur.fetchone()

(1, 'Alice', 1, '2025-04-25', '2025-04-28')

### Mengambil Semua Data `Rooms`
Method `fetchall()` berarti ia mengambil semua entries dari perintah SQL

In [6]:
# Cek data yang sudah dimasukkan ke tabel Rooms
cur.execute("SELECT * FROM Rooms;")
rows = cur.fetchall()
for room_id, number, type, price in rows:
    print(f"Room ID: {room_id}, Number: {number}, Type: {type}, Price: {price}")

Room ID: 1, Number: 101, Type: Single, Price: 500000.0
Room ID: 2, Number: 102, Type: Double, Price: 750000.0
Room ID: 3, Number: 201, Type: Suite, Price: 1500000.0
Room ID: 4, Number: 202, Type: Single, Price: 500000.0
Room ID: 5, Number: 203, Type: Deluxe, Price: 1700000.0


### Mengambil Banyak Data 
Method `fetchmany()` digunakan untuk mengambil beberapa baris data dari perintah SQL sesuai dengan jumlah yang diinginkan.

In [7]:
cur.execute("SELECT * FROM Rooms;")
cur.fetchmany(3)

[(1, '101', 'Single', 500000.0),
 (2, '102', 'Double', 750000.0),
 (3, '201', 'Suite', 1500000.0)]

Kalau parameternya melebihi isi data, maka ya dikembalikan secukupnya.

In [8]:
cur.execute("SELECT * FROM Bookings;")
cur.fetchmany(2)

[(1, 'Alice', 1, '2025-04-25', '2025-04-28')]

### Mengatur Constrain
Mengatur contrian harus membuat koneksi ulang dengan database.

In [9]:
conn = sqlite3.connect("hotel.db")
conn.execute("PRAGMA foreign_keys = ON;")
cur = conn.cursor()

cur.execute("SELECT * FROM Bookings;")
cur.fetchall()

[(1, 'Alice', 1, '2025-04-25', '2025-04-28')]

Apabila kita menambahkan data yang tidak sesuai dengan primary key, maka akan error

In [10]:
try:
    cur.execute(
        "INSERT INTO Bookings (guest_name, room_id, check_in, check_out) VALUES (?, ?, ?, ?)",
        ("Eve", 999, "2025-05-01", "2025-05-03")
    )
    print("Data berhasil dimasukkan ke tabel Bookings")
except sqlite3.IntegrityError as e:
    print("Gagal memasukkan data karena foreign key:", e)


Gagal memasukkan data karena foreign key: FOREIGN KEY constraint failed


In [11]:
try:
    cur.executemany(
        "INSERT INTO Bookings (guest_name, room_id, check_in, check_out) VALUES (?, ?, ?, ?)",
        [
            ("Eve", 3, "2025-05-01", "2025-05-03"),
            ("Bob", 1, "2025-06-01", "2025-06-03"),
            ("Charlie", 2, "2025-07-03", "2025-07-04"),
            ("Dave", 3, "2025-05-02", "2025-05-03")
        ]
    )
    conn.commit()
    print("Data berhasil dimasukkan ke tabel Bookings")
except sqlite3.IntegrityError as e:
    print("Gagal memasukkan data karena foreign key:", e)

Data berhasil dimasukkan ke tabel Bookings


### Coba hapus data
Kita akan menghapus data di tabel `Bookings`. Membuat variable `name` diisi dengan nama pelanggan yang ingin dihapus

In [15]:
name = "Eve"
cur.execute("DELETE FROM Bookings WHERE guest_name = ?;", (name,))

In [16]:
# Cek apakah data dengan guest_name tersebut masih ada
cur.execute("SELECT * FROM Bookings WHERE guest_name = ?;", (name,))
result = cur.fetchone()

if result is None:
    print(f"Tidak ada data dengan nama '{name}' ditemukan — kemungkinan sudah dihapus.")
else:
    print(f"Ada data dengan nama '{name}' — belum dihapus.")

Tidak ada data dengan nama 'Eve' ditemukan — kemungkinan sudah dihapus.


### Menutup koneksi
Setelah menyelesaikan tugas, jangan lupa menutup koneksi database dengan metode `close()` untuk mencegah kebocoran memori, penguncian file, serta memastikan semua perubahan tersimpan dengan aman.

In [17]:
conn.close()